In [1]:
!pip -q install pandas openpyxl networkx scikit-learn plotly



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [1]:
import os
print("Current Working Directory:", os.getcwd())
print("\nFiles in this directory:", os.listdir())

Current Working Directory: /Users/nadan/Documents/projects/demandresponse/data

Files in this directory: ['data vis.ipynb', 'Par_DHD.xlsm', 'hourly_prices.csv', 'Par_DHD_for_code.csv', 'delta.csv', 'Changing_Costs.csv', 'Par_VehicleDetails.xlsx', 'Par_Routes_For_Code.csv', 'Hourly Charge.csv', 'FDL_DHD.xlsm', 'FDL_VehicleDetails.xlsx']


In [4]:
import re
import numpy as np
import pandas as pd
import networkx as nx
from sklearn.manifold import MDS
import plotly.graph_objects as go
from pathlib import Path

VEH_PATH = "Par_VehicleDetails.xlsx"
DHD_PATH = "Par_DHD.xlsm"

DHD_DURATION_COL = "Base Duration"
ALIAS_MAX_MIN = 2

DEPOT_NAME = "PARX"
DEPOT_REF  = 13801

SHOW_DHD_EDGES_UP_TO_MIN = 10

OUT_DIR = Path("bus_outputs")
OUT_DIR.mkdir(exist_ok=True)


def clean_ref(x):
    if pd.isna(x):
        return None
    try:
        return int(float(x))
    except Exception:
        return None

def parse_hhmm(x):
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return None
    s = str(x).strip()
    m = re.match(r"^(\d{1,2}):(\d{2})$", s)
    if not m:
        return None
    return int(m.group(1)) * 60 + int(m.group(2))

def place_key_from_dhd(x):
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s == DEPOT_NAME:
        return DEPOT_NAME
    r = clean_ref(x)
    return str(r) if r is not None else s

def pick_sheet_by_columns(path, required_cols, preferred=None):
    xl = pd.ExcelFile(path)
    names = xl.sheet_names
    if preferred:
        for s in preferred:
            if s in names:
                df = pd.read_excel(path, sheet_name=s)
                if all(c in df.columns for c in required_cols):
                    return df, s
    for s in names:
        df = pd.read_excel(path, sheet_name=s)
        if all(c in df.columns for c in required_cols):
            return df, s
    return pd.read_excel(path, sheet_name=names[0]), names[0]

def route_place_to_key(name, ref):
    nm = None if pd.isna(name) else str(name).strip()
    rf = clean_ref(ref)

    if nm == DEPOT_NAME or rf == DEPOT_REF:
        return DEPOT_NAME

    if rf is not None:
        return str(rf)

    return nm

def pretty_label(node_id, name_set):
    if node_id == DEPOT_NAME:
        return f"{DEPOT_NAME} ({DEPOT_REF})"

    names = [n for n in sorted(name_set) if n and n != "nan"]
    nonnum = [n for n in names if not n.isdigit()]
    pick = min(nonnum, key=len) if nonnum else (names[0] if names else str(node_id))
    return f"{pick} ({node_id})"


veh, _ = pick_sheet_by_columns(
    VEH_PATH,
    ["VehicleTask","From1","Refer.","Start1","To1","Refer.1","Identifier","Route","Direction"],
    preferred=["Data","Sheet1"]
)
dhd, _ = pick_sheet_by_columns(
    DHD_PATH,
    ["Start Place","End Place",DHD_DURATION_COL],
    preferred=["Deadhead","Sheet1"]
)

print("Unique VehicleTask:", veh["VehicleTask"].nunique(dropna=True))
print("Unique Route (line numbers):", veh["Route"].nunique(dropna=True))


parent = {}
def find(a):
    parent.setdefault(a, a)
    if parent[a] != a:
        parent[a] = find(parent[a])
    return parent[a]

def union(a,b):
    ra, rb = find(a), find(b)
    if ra != rb:
        parent[rb] = ra

tmp = dhd[["Start Place","End Place",DHD_DURATION_COL]].dropna().copy()
tmp["a"] = tmp["Start Place"].apply(place_key_from_dhd)
tmp["b"] = tmp["End Place"].apply(place_key_from_dhd)
tmp["w"] = tmp[DHD_DURATION_COL].astype(float)

for _, r in tmp.iterrows():
    a, b, w = r["a"], r["b"], float(r["w"])
    if a is None or b is None:
        continue
    if w <= ALIAS_MAX_MIN:
        union(a, b)

all_places = {x for x in set(tmp["a"]).union(set(tmp["b"])) if x is not None}
groups = {}
for x in all_places:
    groups.setdefault(find(x), set()).add(x)

canon = {}
for _, members in groups.items():
    nums = [m for m in members if m.isdigit()]
    rep = min(nums, key=lambda z: int(z)) if nums else sorted(members, key=lambda s: (len(s), s))[0]
    for m in members:
        canon[m] = rep

def canonical_place(k):
    if k is None:
        return None
    k = str(k).strip()
    if k == DEPOT_NAME:
        return DEPOT_NAME
    return canon.get(k, k)

rep_to_aliases = {}
for alias, rep in canon.items():
    rep_to_aliases.setdefault(rep, set()).add(alias)

rep_to_hubid = {}
hub_rows = []
hid = 0
for rep, aliases in sorted(rep_to_aliases.items(), key=lambda x: (-len(x[1]), x[0])):
    if len(aliases) < 2:
        continue
    rep_to_hubid[rep] = hid
    hub_rows.append({"hub_id": hid, "canonical": rep, "aliases": ";".join(sorted(aliases))})
    hid += 1

pd.DataFrame(hub_rows).to_csv(OUT_DIR / f"hubs_{ALIAS_MAX_MIN}min.csv", index=False)


G = nx.Graph()
for _, r in tmp.iterrows():
    u = canonical_place(r["a"])
    v = canonical_place(r["b"])
    w = float(r["w"])
    if u is None or v is None or u == v:
        continue
    if G.has_edge(u, v):
        if w < G[u][v]["weight"]:
            G[u][v]["weight"] = w
    else:
        G.add_edge(u, v, weight=w)

if DEPOT_NAME not in G:
    G.add_node(DEPOT_NAME)

print("Graph nodes:", G.number_of_nodes(), "edges:", G.number_of_edges())

nodes = list(G.nodes())
idx = {n: i for i, n in enumerate(nodes)}
n = len(nodes)
BIG = 9999.0

D = np.full((n, n), BIG, dtype=float)
np.fill_diagonal(D, 0.0)
for u in nodes:
    i = idx[u]
    lengths = nx.single_source_dijkstra_path_length(G, u, weight="weight")
    for v, dist in lengths.items():
        D[i, idx[v]] = min(D[i, idx[v]], float(dist))
D = np.minimum(D, D.T)

mds = MDS(n_components=2, dissimilarity="precomputed", random_state=42, n_init=4, max_iter=300)
XY = mds.fit_transform(D)
xy = {nodes[i]: {"x": XY[i, 0], "y": XY[i, 1]} for i in range(n)}

edge_x, edge_y = [], []
for u, v, data in G.edges(data=True):
    if data.get("weight", 9999) <= SHOW_DHD_EDGES_UP_TO_MIN:
        edge_x += [xy[u]["x"], xy[v]["x"], None]
        edge_y += [xy[u]["y"], xy[v]["y"], None]

name_map = {}
legs = veh[["From1", "Refer.", "To1", "Refer.1"]].dropna(subset=["From1", "To1"]).copy()
legs["u_key"] = [route_place_to_key(n, r) for n, r in zip(legs["From1"], legs["Refer."])]
legs["v_key"] = [route_place_to_key(n, r) for n, r in zip(legs["To1"], legs["Refer.1"])]
legs["u"] = legs["u_key"].apply(canonical_place)
legs["v"] = legs["v_key"].apply(canonical_place)

for _, row in legs.iterrows():
    name_map.setdefault(row["u"], set()).add(str(row["From1"]).strip())
    name_map.setdefault(row["v"], set()).add(str(row["To1"]).strip())
name_map.setdefault(DEPOT_NAME, set()).add(DEPOT_NAME)


veh_id = veh.copy()
veh_id["Identifier"] = veh_id["Identifier"].astype(str).str.strip().str.lower()
rech = veh_id[veh_id["Identifier"] == "recharge"].copy()

rech_u = [route_place_to_key(n, r) for n, r in zip(rech["From1"], rech["Refer."])]
rech_v = [route_place_to_key(n, r) for n, r in zip(rech["To1"],   rech["Refer.1"])]

charging_nodes = set(map(canonical_place, rech_u)) | set(map(canonical_place, rech_v))
charging_nodes.discard(None)
print("Charging nodes found:", len(charging_nodes))


place_ids = list(xy.keys())
xs = [xy[p]["x"] for p in place_ids]
ys = [xy[p]["y"] for p in place_ids]

hub_color = []
hover = []
deg = []

for p in place_ids:
    hub = rep_to_hubid.get(p, -1)
    hub_color.append(hub)
    deg.append(int(G.degree(p)) if p in G else 0)

    label = pretty_label(p, name_map.get(p, {str(p)}))
    aliases = rep_to_aliases.get(p, set())
    is_ch = (p in charging_nodes)

    hover.append(
        f"{label}"
        + ("<br>⚡ Charging station" if is_ch else "")
        + f"<br>hub_id: {hub}"
        + f"<br>degree: {int(G.degree(p)) if p in G else 0}"
        + f"<br>aliases_in_DHD: {', '.join(sorted(aliases)) if aliases else '-'}"
    )

deg_arr = np.array(deg, dtype=float)
size = 6 + 10 * (deg_arr / (deg_arr.max() if deg_arr.max() > 0 else 1.0))

fig_stops = go.Figure()

fig_stops.add_trace(go.Scatter(
    x=edge_x, y=edge_y,
    mode="lines",
    line=dict(width=1),
    opacity=0.25,
    hoverinfo="skip",
    showlegend=False
))

fig_stops.add_trace(go.Scattergl(
    x=xs, y=ys,
    mode="markers",
    marker=dict(size=size, color=hub_color, showscale=True),
    hovertemplate="%{customdata}<extra></extra>",
    customdata=hover,
    showlegend=False
))

ch_ids = [p for p in place_ids if p in charging_nodes]
ch_x = [xy[p]["x"] for p in ch_ids]
ch_y = [xy[p]["y"] for p in ch_ids]
ch_hover = [pretty_label(p, name_map.get(p, {str(p)})) + "<br>⚡ Charging station" for p in ch_ids]

fig_stops.add_trace(go.Scattergl(
    x=ch_x, y=ch_y,
    mode="markers+text",
    text=["⚡"] * len(ch_ids),
    textposition="middle center",
    marker=dict(size=18, symbol="diamond"),
    hovertemplate="%{customdata}<extra></extra>",
    customdata=ch_hover,
    name="Charging stations",
    showlegend=True
))

fig_stops.update_layout(
    title=f"GLOBAL MAP: all stops | hubs = DHD duration <= {ALIAS_MAX_MIN} min | ⚡ = charging",
    xaxis=dict(visible=False),
    yaxis=dict(visible=False),
    height=900,
    margin=dict(l=10, r=10, t=70, b=10)
)

out1 = OUT_DIR / "global_stops_hubs.html"
fig_stops.write_html(out1, include_plotlyjs="cdn")
print("Saved:", out1)


cols = ["VehicleTask","Identifier","Route","Direction","From1","Refer.","Start1","To1","Refer.1"]
t = veh[cols].dropna(subset=["VehicleTask","From1","To1","Start1"]).copy()
t["t"] = t["Start1"].apply(parse_hhmm)
t = t.dropna(subset=["t"]).sort_values(["VehicleTask", "t"])

t["u_raw"] = [route_place_to_key(n, r) for n, r in zip(t["From1"], t["Refer."])]
t["v_raw"] = [route_place_to_key(n, r) for n, r in zip(t["To1"],   t["Refer.1"])]
t["u"] = t["u_raw"].apply(canonical_place)
t["v"] = t["v_raw"].apply(canonical_place)

vehicle_tasks = sorted(t["VehicleTask"].dropna().unique().tolist())
print("VehicleTasks to draw:", len(vehicle_tasks))

fig_routes = go.Figure()

fig_routes.add_trace(go.Scatter(
    x=edge_x, y=edge_y,
    mode="lines",
    line=dict(width=1),
    opacity=0.15,
    hoverinfo="skip",
    showlegend=False
))

fig_routes.add_trace(go.Scattergl(
    x=xs, y=ys,
    mode="markers",
    marker=dict(size=size, color=hub_color, showscale=True),
    hovertemplate="%{customdata}<extra></extra>",
    customdata=hover,
    showlegend=False
))

fig_routes.add_trace(go.Scattergl(
    x=ch_x, y=ch_y,
    mode="markers+text",
    text=["⚡"] * len(ch_ids),
    textposition="middle center",
    marker=dict(size=18, symbol="diamond"),
    hovertemplate="%{customdata}<extra></extra>",
    customdata=ch_hover,
    name="Charging stations",
    showlegend=True
))

paths_ok = 0
for vt in vehicle_tasks:
    df = t[t["VehicleTask"] == vt]
    seq = [df.iloc[0]["u"]] + df["v"].tolist()

    expanded = []
    ok = True
    for a, b in zip(seq[:-1], seq[1:]):
        if a not in G or b not in G:
            ok = False
            break
        try:
            path = nx.shortest_path(G, a, b, weight="weight")
        except nx.NetworkXNoPath:
            ok = False
            break
        expanded = path if not expanded else (expanded + path[1:])

    expanded = [p for p in expanded if p in xy]
    if len(expanded) < 2:
        ok = False

    if ok:
        paths_ok += 1

    px = [xy[p]["x"] for p in expanded] if ok else []
    py = [xy[p]["y"] for p in expanded] if ok else []

    fig_routes.add_trace(go.Scatter(
        x=px, y=py,
        mode="lines",
        line=dict(width=5),
        opacity=0.65,
        hoverinfo="skip",
        name=f"VehicleTask {vt}",
        visible=False
    ))

print("VehicleTask paths built:", paths_ok, "out of", len(vehicle_tasks))

# Dropdown buttons
N = len(vehicle_tasks)
base_traces = 3  # (edges, stops, chargers)

buttons = []

# none
vis_none = [True]*base_traces + [False]*N
buttons.append(dict(
    label="Show no routes",
    method="update",
    args=[{"visible": vis_none},
          {"title": "GLOBAL ROUTES: stops + ⚡ only"}]
))

# all
vis_all = [True]*base_traces + [True]*N
buttons.append(dict(
    label="Show ALL VehicleTasks",
    method="update",
    args=[{"visible": vis_all},
          {"title": "GLOBAL ROUTES: ALL VehicleTasks"}]
))

# one-by-one
for i, vt in enumerate(vehicle_tasks):
    vis_one = [True]*base_traces + [False]*N
    vis_one[base_traces + i] = True
    buttons.append(dict(
        label=f"VehicleTask {vt}",
        method="update",
        args=[{"visible": vis_one},
              {"title": f"GLOBAL ROUTES: VehicleTask {vt} (⚡ marked)"}]
    ))

fig_routes.update_layout(
    title="GLOBAL ROUTES: (use dropdown)",
    xaxis=dict(visible=False),
    yaxis=dict(visible=False),
    height=900,
    margin=dict(l=10, r=10, t=70, b=10),
    updatemenus=[dict(
        type="dropdown",
        x=0.02, y=0.98,
        xanchor="left", yanchor="top",
        buttons=buttons
    )]
)

out2 = OUT_DIR / "global_routes_dropdown.html"
fig_routes.write_html(out2, include_plotlyjs="cdn")
print("Saved:", out2)

print("DONE. Open:")
print(" -", out1)
print(" -", out2)


Unique VehicleTask: 42
Unique Route (line numbers): 10
Graph nodes: 172 edges: 1205
Charging nodes found: 6
Saved: bus_outputs/global_stops_hubs.html
VehicleTasks to draw: 42
VehicleTask paths built: 42 out of 42
Saved: bus_outputs/global_routes_dropdown.html
DONE. Open:
 - bus_outputs/global_stops_hubs.html
 - bus_outputs/global_routes_dropdown.html


In [7]:
import re
import numpy as np
import pandas as pd
import networkx as nx
from sklearn.manifold import MDS
import plotly.graph_objects as go
from pathlib import Path


VEH_PATH = "Par_VehicleDetails.xlsx"
DHD_PATH = "Par_DHD.xlsm"

DHD_DURATION_COL = "Base Duration"
ALIAS_MAX_MIN = 2

DEPOT_NAME = "PARX"
DEPOT_REF  = 13801

SHOW_DHD_EDGES_UP_TO_MIN = 10

OUT_DIR = Path("bus_outputs")
OUT_DIR.mkdir(exist_ok=True)


def clean_ref(x):
    if pd.isna(x): return None
    try: return int(float(x))
    except: return None

def parse_hhmm(x):
    if x is None or (isinstance(x, float) and np.isnan(x)): return None
    s = str(x).strip()
    m = re.match(r"^(\d{1,2}):(\d{2})$", s)
    if not m: return None
    return int(m.group(1))*60 + int(m.group(2))

def place_key_from_dhd(x):
    if pd.isna(x): return None
    s = str(x).strip()
    if s == DEPOT_NAME: return DEPOT_NAME
    r = clean_ref(x)
    return str(r) if r is not None else s

def pick_sheet_by_columns(path, required_cols, preferred=None):
    xl = pd.ExcelFile(path)
    names = xl.sheet_names
    if preferred:
        for s in preferred:
            if s in names:
                df = pd.read_excel(path, sheet_name=s)
                if all(c in df.columns for c in required_cols):
                    return df, s
    for s in names:
        df = pd.read_excel(path, sheet_name=s)
        if all(c in df.columns for c in required_cols):
            return df, s
    return pd.read_excel(path, sheet_name=names[0]), names[0]

def route_place_to_key(name, ref):
    nm = None if pd.isna(name) else str(name).strip()
    rf = clean_ref(ref)
    if nm == DEPOT_NAME or rf == DEPOT_REF:
        return DEPOT_NAME
    if rf is not None:
        return str(rf)
    return nm

def pretty_label(node_id, name_set):
    if node_id == DEPOT_NAME:
        return f"{DEPOT_NAME} ({DEPOT_REF})"
    names = [n for n in sorted(name_set) if n and n != "nan"]
    nonnum = [n for n in names if not n.isdigit()]
    pick = min(nonnum, key=len) if nonnum else (names[0] if names else str(node_id))
    return f"{pick} ({node_id})"


veh, _ = pick_sheet_by_columns(
    VEH_PATH,
    ["VehicleTask","Identifier","Route","Direction","From1","Refer.","Start1","To1","Refer.1"],
    preferred=["Data","Sheet1"]
)
dhd, _ = pick_sheet_by_columns(
    DHD_PATH,
    ["Start Place","End Place",DHD_DURATION_COL],
    preferred=["Deadhead","Sheet1"]
)

veh = veh.copy()
veh["Route"] = pd.to_numeric(veh["Route"], errors="coerce")
veh["Direction"] = pd.to_numeric(veh["Direction"], errors="coerce")

print("Unique line routes (Route):", veh["Route"].nunique(dropna=True))


parent = {}
def find(a):
    parent.setdefault(a, a)
    if parent[a] != a:
        parent[a] = find(parent[a])
    return parent[a]
def union(a,b):
    ra, rb = find(a), find(b)
    if ra != rb:
        parent[rb] = ra

tmp = dhd[["Start Place","End Place",DHD_DURATION_COL]].dropna().copy()
tmp["a"] = tmp["Start Place"].apply(place_key_from_dhd)
tmp["b"] = tmp["End Place"].apply(place_key_from_dhd)
tmp["w"] = tmp[DHD_DURATION_COL].astype(float)

for _, r in tmp.iterrows():
    a,b,w = r["a"], r["b"], float(r["w"])
    if a is None or b is None:
        continue
    if w <= ALIAS_MAX_MIN:
        union(a,b)

all_places = {x for x in set(tmp["a"]).union(set(tmp["b"])) if x is not None}
groups = {}
for x in all_places:
    groups.setdefault(find(x), set()).add(x)

canon = {}
for _, members in groups.items():
    nums = [m for m in members if m.isdigit()]
    rep = min(nums, key=lambda z:int(z)) if nums else sorted(members, key=lambda s:(len(s),s))[0]
    for m in members:
        canon[m] = rep

def canonical_place(k):
    if k is None: return None
    k = str(k).strip()
    if k == DEPOT_NAME: return DEPOT_NAME
    return canon.get(k, k)

G = nx.Graph()
for _, r in tmp.iterrows():
    u = canonical_place(r["a"])
    v = canonical_place(r["b"])
    w = float(r["w"])
    if u is None or v is None or u == v:
        continue
    if G.has_edge(u,v):
        if w < G[u][v]["weight"]:
            G[u][v]["weight"] = w
    else:
        G.add_edge(u,v,weight=w)

if DEPOT_NAME not in G:
    G.add_node(DEPOT_NAME)

nodes = list(G.nodes())
idx = {n:i for i,n in enumerate(nodes)}
n = len(nodes)
BIG = 9999.0

D = np.full((n,n), BIG, dtype=float)
np.fill_diagonal(D, 0.0)
for u in nodes:
    i = idx[u]
    lengths = nx.single_source_dijkstra_path_length(G, u, weight="weight")
    for v, dist in lengths.items():
        D[i, idx[v]] = min(D[i, idx[v]], float(dist))
D = np.minimum(D, D.T)

mds = MDS(n_components=2, dissimilarity="precomputed", random_state=42, n_init=4, max_iter=300)
XY = mds.fit_transform(D)
xy = {nodes[i]: {"x": XY[i,0], "y": XY[i,1]} for i in range(n)}

edge_x, edge_y = [], []
for u, v, data in G.edges(data=True):
    if data.get("weight", 9999) <= SHOW_DHD_EDGES_UP_TO_MIN:
        edge_x += [xy[u]["x"], xy[v]["x"], None]
        edge_y += [xy[u]["y"], xy[v]["y"], None]

name_map = {}
legs = veh[["From1","Refer.","To1","Refer.1"]].dropna(subset=["From1","To1"]).copy()
legs["u_key"] = [route_place_to_key(n,r) for n,r in zip(legs["From1"], legs["Refer."])]
legs["v_key"] = [route_place_to_key(n,r) for n,r in zip(legs["To1"],   legs["Refer.1"])]
legs["u"] = legs["u_key"].apply(canonical_place)
legs["v"] = legs["v_key"].apply(canonical_place)

for _, row in legs.iterrows():
    name_map.setdefault(row["u"], set()).add(str(row["From1"]).strip())
    name_map.setdefault(row["v"], set()).add(str(row["To1"]).strip())
name_map.setdefault(DEPOT_NAME, set()).add(DEPOT_NAME)

place_ids = list(xy.keys())
xs = [xy[p]["x"] for p in place_ids]
ys = [xy[p]["y"] for p in place_ids]

veh_id = veh.copy()
veh_id["Identifier"] = veh_id["Identifier"].astype(str).str.strip().str.lower()
rech = veh_id[veh_id["Identifier"] == "recharge"].copy()

rech_u = [route_place_to_key(n, r) for n, r in zip(rech["From1"], rech["Refer."])]
rech_v = [route_place_to_key(n, r) for n, r in zip(rech["To1"],   rech["Refer.1"])]

charging_nodes = set(map(canonical_place, rech_u)) | set(map(canonical_place, rech_v))
charging_nodes.discard(None)
print("Charging nodes found:", len(charging_nodes))

hover = []
for p in place_ids:
    label = pretty_label(p, name_map.get(p, {str(p)}))
    if p in charging_nodes:
        label += "<br>⚡ Charging station"
    hover.append(label)

ch_ids = [p for p in place_ids if p in charging_nodes]
ch_x = [xy[p]["x"] for p in ch_ids]
ch_y = [xy[p]["y"] for p in ch_ids]
ch_hover = [pretty_label(p, name_map.get(p, {str(p)})) + "<br>⚡ Charging station" for p in ch_ids]


reg = veh.copy()
reg["Identifier"] = reg["Identifier"].astype(str).str.strip().str.lower()
reg = reg[reg["Identifier"] == "regular"]

reg = reg.dropna(subset=["Route","Direction","From1","To1","Refer.","Refer.1","Start1"])
reg["t"] = reg["Start1"].apply(parse_hhmm)
reg = reg.dropna(subset=["t"])

reg["u_raw"] = [route_place_to_key(n,r) for n,r in zip(reg["From1"], reg["Refer."])]
reg["v_raw"] = [route_place_to_key(n,r) for n,r in zip(reg["To1"],   reg["Refer.1"])]
reg["u"] = reg["u_raw"].apply(canonical_place)
reg["v"] = reg["v_raw"].apply(canonical_place)

counts = (
    reg.groupby(["Route","Direction","u","v"])
       .size()
       .reset_index(name="cnt")
)

route_keys = sorted(counts[["Route","Direction"]].drop_duplicates().itertuples(index=False, name=None))


fig = go.Figure()

# background
fig.add_trace(go.Scatter(
    x=edge_x, y=edge_y,
    mode="lines",
    line=dict(width=1),
    opacity=0.15,
    hoverinfo="skip",
    showlegend=False
))

# all stops
fig.add_trace(go.Scattergl(
    x=xs, y=ys,
    mode="markers",
    marker=dict(size=6),
    hovertemplate="%{customdata}<extra></extra>",
    customdata=hover,
    showlegend=False
))

# chargers overlay (always visible)
fig.add_trace(go.Scattergl(
    x=ch_x, y=ch_y,
    mode="markers+text",
    text=["⚡"] * len(ch_ids),
    textposition="middle center",
    marker=dict(size=18, symbol="diamond"),
    hovertemplate="%{customdata}<extra></extra>",
    customdata=ch_hover,
    name="Charging stations",
    showlegend=True
))

# one trace per (Route,Direction) as “shape”
route_traces = []
for (rt, dr) in route_keys:
    sub = counts[(counts["Route"] == rt) & (counts["Direction"] == dr)].copy()
    if sub.empty:
        continue

    cmin, cmax = sub["cnt"].min(), sub["cnt"].max()
    denom = (cmax - cmin) if cmax > cmin else 1.0

    ex, ey = [], []
    widths = []

    for _, r in sub.iterrows():
        u, v, c = r["u"], r["v"], int(r["cnt"])
        if u not in xy or v not in xy or u == v:
            continue
        ex += [xy[u]["x"], xy[v]["x"], None]
        ey += [xy[u]["y"], xy[v]["y"], None]
        widths.append(2 + 6 * ((c - cmin) / denom))

    mean_w = float(np.mean(widths)) if widths else 3.0

    fig.add_trace(go.Scatter(
        x=ex, y=ey,
        mode="lines",
        line=dict(width=mean_w),
        opacity=0.75,
        hoverinfo="skip",
        name=f"Route {int(rt)} Dir {int(dr)}",
        visible=True#False
    ))
    route_traces.append((rt, dr))

# dropdown
N = len(route_traces)
base_traces = 3  # background, stops, chargers

buttons = []
buttons.append(dict(
    label="Show no line",
    method="update",
    args=[{"visible": [True]*base_traces + [False]*N},
          {"title": "LINE ROUTES: stops + ⚡ only"}]
))
buttons.append(dict(
    label="Show ALL lines",
    method="update",
    args=[{"visible": [True]*base_traces + [True]*N},
          {"title": "LINE ROUTES: ALL lines (⚡ marked)"}]
))
for i, (rt, dr) in enumerate(route_traces):
    vis = [True]*base_traces + [False]*N
    vis[base_traces + i] = True
    buttons.append(dict(
        label=f"Route {int(rt)} Dir {int(dr)}",
        method="update",
        args=[{"visible": vis},
              {"title": f"LINE ROUTE {int(rt)} | Direction {int(dr)} (⚡ marked)"}]
    ))

fig.update_layout(
    title="LINE ROUTES (Regular legs): use dropdown",
    xaxis=dict(visible=False),
    yaxis=dict(visible=False),
    height=900,
    margin=dict(l=10, r=10, t=70, b=10),
    updatemenus=[dict(
        type="dropdown",
        x=0.02, y=0.98,
        xanchor="left", yanchor="top",
        buttons=buttons
    )]
)

out = OUT_DIR / "lines_routes_dropdown.html"
fig.write_html(out, include_plotlyjs="cdn")
print("Saved:", out)
print("Open it:")
print(" -", out)


Unique line routes (Route): 10
Charging nodes found: 6
Saved: bus_outputs/lines_routes_dropdown.html
Open it:
 - bus_outputs/lines_routes_dropdown.html


In [6]:
# --- DEBUGGING SNIPPET START ---
print(f"Total nodes in Map (from DHD file): {len(xy)}")
print(f"Total legs in Schedule (Regular): {len(reg)}")

# Check if the stops in the schedule exist in the map
schedule_stops = set(reg["u"]).union(set(reg["v"]))
missing_stops = [s for s in schedule_stops if s not in xy]

print(f"Unique stops in Schedule: {len(schedule_stops)}")
print(f"Stops MISSING from Map: {len(missing_stops)}")

if len(missing_stops) > 0:
    print("\nExamples of missing stops (Present in Schedule, missing in DHD Map):")
    print(missing_stops[:10])
    
    print("\nExamples of keys that ARE in the map:")
    print(list(xy.keys())[:10])
else:
    print("\nAll stops in schedule found in map. The issue is likely elsewhere.")
# --- DEBUGGING SNIPPET END ---

Total nodes in Map (from DHD file): 172
Total legs in Schedule (Regular): 651
Unique stops in Schedule: 5
Stops MISSING from Map: 0

All stops in schedule found in map. The issue is likely elsewhere.
